In [2]:
from keras_utilities import *
from keras_utilities.models.vgg16bn import Vgg16BN
from keras_utilities.models.vgg16 import Vgg16
from keras.layers import Dense
import os
from numpy import array
from IPython.display import FileLink
from numpy.random import permutation
from keras.preprocessing import image
from keras.applications import ResNet50
from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Input, Dense, Flatten
from keras import backend as K
from keras.optimizers import Adam

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5103)
Using Theano backend.


In [3]:
prod = True

if not prod:
    path = 'sample/'
else:
    path = 'data/'
    
batch_size = 64

model_dir = "models/"
results_dir = "results/"

In [4]:
batches = get_batches(path+"train", batch_size=64)
val_batches = get_batches(path+"valid", batch_size=64)

Found 17778 images belonging to 10 classes.
Found 4646 images belonging to 10 classes.


## Epoch 1

In [10]:
vgg16bn = Vgg16BN()
vgg16bn.ft(10)
vgg16bn.compile(lr=.0001)
vgg16bn.fit(batches=batches, val_batches=val_batches)

Epoch 1/1
17778/17778 [==============================] - 551s - loss: 4.0458 - acc: 0.2536 - val_loss: 3.2759 - val_acc: 0.2923


## Epoch 2

In [11]:
vgg16bn.fit(batches=batches, val_batches=val_batches)

Epoch 1/1
17778/17778 [==============================] - 552s - loss: 2.1896 - acc: 0.4893 - val_loss: 4.0625 - val_acc: 0.2796


## Higher Learning Rate

In [12]:
vgg16bn = Vgg16BN()
vgg16bn.ft(10)
vgg16bn.compile(lr=.01)
vgg16bn.fit(batches=batches, val_batches=val_batches)

Epoch 1/1
17778/17778 [==============================] - 551s - loss: 4.0452 - acc: 0.6693 - val_loss: 9.6944 - val_acc: 0.3362


## ResNet

In [13]:
rn = ResNet50(); rn.layers.pop(); rn.layers.pop(); rn.layers.pop()

In [14]:
x = rn.layers[-1].output

In [15]:
mod = AveragePooling2D(pool_size=(7,7))(x)
mod = Flatten()(mod)
mod = Dense(10, activation='softmax')(mod)

In [16]:
newmod = Model(input=rn.input, output=mod)

In [17]:
opt = Adam(lr=.0001)
newmod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
newmod.fit_generator(batches, 
                     batches.nb_sample, 
                     validation_data=val_batches, 
                     nb_val_samples = val_batches.nb_sample, 
                     nb_epoch=1)

KeyboardInterrupt: 

In [24]:
def finetune_resnet(nb_classes, lr=.001):
    rn = ResNet50(); rn.layers.pop(); rn.layers.pop(); rn.layers.pop()
    
    x = rn.layers[-1].output 
    mod = AveragePooling2D(name='avg_pool_ft', pool_size=(7,7))(x)
    mod = Flatten(name='flatten_ft')(mod)
    mod = Dense(nb_classes, name='dense_ft', activation='softmax')(mod)
    
    newmod = Model(input=rn.input, output=mod)
    
    for i in newmod.layers:
        if i.name in ['avg_pool_ft', 'flatten_ft', 'dense_ft']:
            i.trainable = True
        else:
            i.trainable = False
    
    opt = Adam(lr=lr)
    newmod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
            
    return newmod
    

In [25]:
res_ft = finetune_resnet(10)

In [26]:
res_ft.fit_generator(batches, 
                     batches.nb_sample, 
                     validation_data=val_batches, 
                     nb_val_samples = val_batches.nb_sample, 
                     nb_epoch=1)

Epoch 1/1
17778/17778 [==============================] - 276s - loss: 0.9162 - val_loss: 1.8732


In [40]:
def create_prediction_file(filename, preds=None, model=None):
    if not preds:
        num_test_files = len(os.listdir("data/test/unknown"))
        test_gen = get_batches("data/test")
        preds = model.predict_generator(test_gen, val_samples=num_test_files)
    elif not model and not preds:
        raise ValueError("Must pass either preds or a model.")
        
    files = os.listdir("data/test/unknown/")
        
    pred_str = map(lambda x: ",".join(["%.10f" % i for i in x]), preds[1].tolist())
    rows = [",".join([a,b]) + "\n" for a,b in zip(files, pred_str)]
    with open(filename, "a") as f:
        f.write("img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n")
        for row in rows:
            f.write(row)

In [41]:
??res_ft.predict_generator

In [42]:
create_prediction_file("resnet_last3_ft.csv", model=res_ft)

Found 79726 images belonging to 1 classes.


TypeError: 'float' object is not iterable

In [44]:
num_test_files = len(os.listdir("data/test/unknown"))
test_gen = get_batches("data/test")
preds = res_ft.predict_generator(test_gen, val_samples=num_test_files)

Found 79726 images belonging to 1 classes.


In [45]:
files = os.listdir("data/test/unknown/")

In [46]:
files[:10]

['img_81601.jpg',
 'img_14887.jpg',
 'img_62885.jpg',
 'img_45125.jpg',
 'img_22633.jpg',
 'img_45660.jpg',
 'img_88962.jpg',
 'img_92987.jpg',
 'img_26932.jpg',
 'img_57972.jpg']

In [56]:
filename = "results/resnet_3layer_ft.csv"
pred_str = map(lambda x: ",".join(["%.10f" % i for i in x]), preds.tolist())
rows = [",".join([a,b]) + "\n" for a,b in zip(files, pred_str)]
with open(filename, "a") as f:
    f.write("img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n")
    for row in rows:
        f.write(row)

In [57]:
FileLink("results/resnet_3layer_ft.csv")

/home/ubuntu/statefarm-kaggle/results/resnet_3layer_ft.csv